<a href="https://colab.research.google.com/github/atuberville/FinalProject/blob/cf_machine_learning/college_football_ml_oneTeam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# College Football Machine Learning 2008-2013

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV and Perform Basic Data Cleaning

In [5]:
# Load the data
from google.colab import files
uploaded = files.upload()

file_name='goldcopyhalfstats.csv'
#df = pd.read_csv(file_name, skiprows=0)[:-2]
df=pd.read_csv(file_name)

Saving goldcopyhalfstats.csv to goldcopyhalfstats.csv


In [6]:
df.head()

,year,game_code,a_team_name,a_name_conference,a_team_code,a_result,a_points,b_team_code,team_name,name_conference,b_result,b_points,half_leader,a_rush,b_rush,a_offense_rush_yards,b_defense_rush_yards,a_defense_rush_yards,b_offense_rush_yards,a_offense_pass_yards,b_defense_pass_yards,a_defense_pass_yards,b_offense_pass_yards,a_pass,b_pass,a_incomplete_pass,b_incomplete_pass,a_penalty,b_penalty,a_penalty_yards,b_penalty_yards,half_diff_score,half_diff_score_abs,visit_date,stadium_code,stadium_name,site,city,state,capacity,surface,year_opened
0,2008,5068820080906,Syracuse ...,Atlantic Coast Conference ...,688,0,14,5,Akron ...,Mid-American Conference ...,1,28,BTEAM,17,24,84,84,129,129,40,40,154,154,10,15,3,3,2,0,10,0,-14,14,9/6/2008,92,Carrier Dome ...,TEAM,Syracuse ...,NY,49250,FieldTurf,1980
1,2008,8014720080830,Clemson ...,Atlantic Coast Conference ...,147,0,3,8,Alabama ...,Southeastern Conference ...,1,20,BTEAM,9,27,45,45,123,123,56,56,116,116,13,18,6,4,4,4,20,28,-17,17,8/30/2008,38,Georgia Dome ...,NEUTRAL,Atlanta ...,GA,71250,FieldTurf,1992
2,2010,8019320100918,Duke ...,Atlantic Coast Conference ...,193,0,13,8,Alabama ...,Southeastern Conference ...,1,42,BTEAM,16,15,90,90,157,157,129,129,139,139,27,18,12,4,3,2,25,10,-29,29,9/18/2010,80,Wallace Wade Stadium ...,TEAM,Durham ...,NC,33941,Grass,1929
3,2011,27074220110903,Virginia Tech ...,Atlantic Coast Conference ...,742,1,38,27,Appalachian State ...,Southern ...,0,0,ATEAM,25,16,136,136,91,91,140,140,71,71,15,12,8,5,1,2,5,10,38,38,9/3/2011,136,Lane Stadium ...,TEAM,Blacksburg ...,VA,66233,Grass,1965
4,2011,37014720110917,Clemson ...,Atlantic Coast Conference ...,147,1,21,37,Auburn ...,Southeastern Conference ...,0,21,TIED,16,27,61,61,157,157,165,165,125,125,24,15,8,7,4,2,35,20,0,0,9/17/2011,112,Memorial Stadium ...,TEAM,Clemson ...,SC,86400,Grass,1942


In [17]:
# Drop the null columns where all values are null
df2 = df.dropna(axis='columns', how='all').copy()

# Drop the null rows
df2 = df2.dropna()

df2.reset_index(inplace=True, drop=True)

print(len(df2))
df2.head()

4789


,year,game_code,a_team_name,a_name_conference,a_team_code,a_result,a_points,b_team_code,team_name,name_conference,b_result,b_points,half_leader,a_rush,b_rush,a_offense_rush_yards,b_defense_rush_yards,a_defense_rush_yards,b_offense_rush_yards,a_offense_pass_yards,b_defense_pass_yards,a_defense_pass_yards,b_offense_pass_yards,a_pass,b_pass,a_incomplete_pass,b_incomplete_pass,a_penalty,b_penalty,a_penalty_yards,b_penalty_yards,half_diff_score,half_diff_score_abs,visit_date,stadium_code,stadium_name,site,city,state,capacity,surface,year_opened
0,2008,5068820080906,Syracuse ...,Atlantic Coast Conference ...,688,0,14,5,Akron ...,Mid-American Conference ...,1,28,BTEAM,17,24,84,84,129,129,40,40,154,154,10,15,3,3,2,0,10,0,-14,14,9/6/2008,92,Carrier Dome ...,TEAM,Syracuse ...,NY,49250,FieldTurf,1980
1,2008,8014720080830,Clemson ...,Atlantic Coast Conference ...,147,0,3,8,Alabama ...,Southeastern Conference ...,1,20,BTEAM,9,27,45,45,123,123,56,56,116,116,13,18,6,4,4,4,20,28,-17,17,8/30/2008,38,Georgia Dome ...,NEUTRAL,Atlanta ...,GA,71250,FieldTurf,1992
2,2010,8019320100918,Duke ...,Atlantic Coast Conference ...,193,0,13,8,Alabama ...,Southeastern Conference ...,1,42,BTEAM,16,15,90,90,157,157,129,129,139,139,27,18,12,4,3,2,25,10,-29,29,9/18/2010,80,Wallace Wade Stadium ...,TEAM,Durham ...,NC,33941,Grass,1929
3,2011,27074220110903,Virginia Tech ...,Atlantic Coast Conference ...,742,1,38,27,Appalachian State ...,Southern ...,0,0,ATEAM,25,16,136,136,91,91,140,140,71,71,15,12,8,5,1,2,5,10,38,38,9/3/2011,136,Lane Stadium ...,TEAM,Blacksburg ...,VA,66233,Grass,1965
4,2011,37014720110917,Clemson ...,Atlantic Coast Conference ...,147,1,21,37,Auburn ...,Southeastern Conference ...,0,21,TIED,16,27,61,61,157,157,165,165,125,125,24,15,8,7,4,2,35,20,0,0,9/17/2011,112,Memorial Stadium ...,TEAM,Clemson ...,SC,86400,Grass,1942


In [22]:
df3=df2.copy()
df3["result"]=df3['a_result']
df3=df3.drop(['a_result', 'b_result'], axis=1)

df3=df3.drop(['a_defense_rush_yards','b_defense_rush_yards','a_defense_pass_yards','b_defense_pass_yards'],axis=1)

df3.head()

,year,game_code,a_team_name,a_name_conference,a_team_code,a_points,b_team_code,team_name,name_conference,b_points,half_leader,a_rush,b_rush,a_offense_rush_yards,b_offense_rush_yards,a_offense_pass_yards,b_offense_pass_yards,a_pass,b_pass,a_incomplete_pass,b_incomplete_pass,a_penalty,b_penalty,a_penalty_yards,b_penalty_yards,half_diff_score,half_diff_score_abs,visit_date,stadium_code,stadium_name,site,city,state,capacity,surface,year_opened,result
0,2008,5068820080906,Syracuse ...,Atlantic Coast Conference ...,688,14,5,Akron ...,Mid-American Conference ...,28,BTEAM,17,24,84,129,40,154,10,15,3,3,2,0,10,0,-14,14,9/6/2008,92,Carrier Dome ...,TEAM,Syracuse ...,NY,49250,FieldTurf,1980,0
1,2008,8014720080830,Clemson ...,Atlantic Coast Conference ...,147,3,8,Alabama ...,Southeastern Conference ...,20,BTEAM,9,27,45,123,56,116,13,18,6,4,4,4,20,28,-17,17,8/30/2008,38,Georgia Dome ...,NEUTRAL,Atlanta ...,GA,71250,FieldTurf,1992,0
2,2010,8019320100918,Duke ...,Atlantic Coast Conference ...,193,13,8,Alabama ...,Southeastern Conference ...,42,BTEAM,16,15,90,157,129,139,27,18,12,4,3,2,25,10,-29,29,9/18/2010,80,Wallace Wade Stadium ...,TEAM,Durham ...,NC,33941,Grass,1929,0
3,2011,27074220110903,Virginia Tech ...,Atlantic Coast Conference ...,742,38,27,Appalachian State ...,Southern ...,0,ATEAM,25,16,136,91,140,71,15,12,8,5,1,2,5,10,38,38,9/3/2011,136,Lane Stadium ...,TEAM,Blacksburg ...,VA,66233,Grass,1965,1
4,2011,37014720110917,Clemson ...,Atlantic Coast Conference ...,147,21,37,Auburn ...,Southeastern Conference ...,21,TIED,16,27,61,157,165,125,24,15,8,7,4,2,35,20,0,0,9/17/2011,112,Memorial Stadium ...,TEAM,Clemson ...,SC,86400,Grass,1942,1


#Filter Data

In [31]:
df_filter=df3.copy()

# Throw out "blowout" games
blowoutThresh=14
df_filter=df_filter[df_filter['half_diff_score_abs'] <= blowoutThresh]

# reset index
df_filter.reset_index(inplace=True, drop=True)

print(len(df_filter))
df_filter.head(5)

3242


,year,game_code,a_team_name,a_name_conference,a_team_code,a_points,b_team_code,team_name,name_conference,b_points,half_leader,a_rush,b_rush,a_offense_rush_yards,b_offense_rush_yards,a_offense_pass_yards,b_offense_pass_yards,a_pass,b_pass,a_incomplete_pass,b_incomplete_pass,a_penalty,b_penalty,a_penalty_yards,b_penalty_yards,half_diff_score,half_diff_score_abs,visit_date,stadium_code,stadium_name,site,city,state,capacity,surface,year_opened,result
0,2008,5068820080906,Syracuse ...,Atlantic Coast Conference ...,688,14,5,Akron ...,Mid-American Conference ...,28,BTEAM,17,24,84,129,40,154,10,15,3,3,2,0,10,0,-14,14,9/6/2008,92,Carrier Dome ...,TEAM,Syracuse ...,NY,49250,FieldTurf,1980,0
1,2011,37014720110917,Clemson ...,Atlantic Coast Conference ...,147,21,37,Auburn ...,Southeastern Conference ...,21,TIED,16,27,61,157,165,125,24,15,8,7,4,2,35,20,0,0,9/17/2011,112,Memorial Stadium ...,TEAM,Clemson ...,SC,86400,Grass,1942,1
2,2013,37023420140106,Florida State ...,Atlantic Coast Conference ...,234,10,37,Auburn ...,Southeastern Conference ...,21,BTEAM,18,29,137,131,62,107,15,13,10,6,2,3,20,15,-11,11,1/6/2014,15,Rose Bowl ...,NEUTRAL,Pasadena ...,CA,91136,Grass,1922,1
3,2009,51074920090905,Wake Forest ...,Atlantic Coast Conference ...,749,7,51,Baylor ...,Big 12 Conference ...,10,BTEAM,21,14,68,43,44,107,6,16,1,4,5,2,40,10,-3,3,9/5/2009,83,Groves Stadium ...,TEAM,Winston Salem ...,NC,31500,FieldTurf,1968,0
4,2011,61041520111001,Miami (Florida) ...,Atlantic Coast Conference ...,415,14,61,Bethune-Cookman ...,Mid-Eastern ...,7,ATEAM,9,24,-22,-6,107,89,9,20,2,9,3,6,28,40,7,7,10/1/2011,30,Sun Life Stadium ...,TEAM,Miami ...,FL,75235,Grass,1987,1


In [ ]:
df_filter=df2.copy()
df_filter=df_filter[abs(df_filter['OFFENSE POINTS'] - df_filter['DEFENSE POINTS']) <= blowoutThresh]

df_filter.reset_index(inplace=True, drop=True)

#df_filter['DELTA SCORE']=df_filter['OFFENSE POINTS']-df_filter["DEFENSE POINTS"]
print(len(df_filter))
df_filter.head(5)

In [32]:
df_filter.columns

Index(['year', 'game_code', 'a_team_name', 'a_name_conference', 'a_team_code',
       'a_points', 'b_team_code', 'team_name', 'name_conference', 'b_points',
       'half_leader', 'a_rush', 'b_rush', 'a_offense_rush_yards',
       'b_offense_rush_yards', 'a_offense_pass_yards', 'b_offense_pass_yards',
       'a_pass', 'b_pass', 'a_incomplete_pass', 'b_incomplete_pass',
       'a_penalty', 'b_penalty', 'a_penalty_yards', 'b_penalty_yards',
       'half_diff_score', 'half_diff_score_abs', 'visit_date', 'stadium_code',
       'stadium_name', 'site', 'city', 'state', 'capacity', 'surface',
       'year_opened', 'result'],
      dtype='object')

# Split the Data into Training and Testing and Scale

In [113]:
# Create our features
X = df_filter.copy()
X=X.drop({'year', 'game_code', 'a_team_name', 'a_name_conference', 'a_team_code',
          'b_team_code', 'team_name', 'name_conference','half_leader',
          'half_diff_score', 'half_diff_score_abs', 'visit_date', 'stadium_code',
          'stadium_name', 'site', 'city', 'state', 'capacity',
          'year_opened', 'result'},axis=1)

# Need to convert columns with strings to numeric
# Get column names that are non-numeric
column_names=X.select_dtypes(exclude=[np.number]).columns
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for columnName in column_names:
  X[columnName]=le.fit_transform(X[columnName])

# Create our target
y = df_filter['result'].copy()

X.head()

,a_points,b_points,a_rush,b_rush,a_offense_rush_yards,b_offense_rush_yards,a_offense_pass_yards,b_offense_pass_yards,a_pass,b_pass,a_incomplete_pass,b_incomplete_pass,a_penalty,b_penalty,a_penalty_yards,b_penalty_yards,surface
0,14,28,17,24,84,129,40,154,10,15,3,3,2,0,10,0,3
1,21,21,16,27,61,157,165,125,24,15,8,7,4,2,35,20,4
2,10,21,18,29,137,131,62,107,15,13,10,6,2,3,20,15,4
3,7,10,21,14,68,43,44,107,6,16,1,4,5,2,40,10,3
4,14,7,9,24,-22,-6,107,89,9,20,2,9,3,6,28,40,4


In [114]:
# Check the balance of our target values
abs(X['a_points']-X['b_points']).value_counts()

# We have a lot of games with a 7-point spread at half compared to the others (makes sense)
# May want to look at stratifying?  So that our model predicts well at any spread

7     516
3     397
14    375
10    343
0     330
4     319
11    234
1     145
6     138
13    137
8     106
2      59
9      58
5      52
12     33
dtype: int64

# Sampling Methods and Scale


In [115]:

from sklearn.model_selection import train_test_split

# stratify by point spread
stratify_values=abs(df_filter['a_points']-df_filter['b_points'])
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    stratify=stratify_values, 
                                                    test_size=0.5)

# Scale the Data
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

#scaler = StandardScaler()
scaler=MinMaxScaler()
X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
X_train_scaled = X_train
X_test_scaled = X_test


# Models

## Import Dependencies

In [116]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier


from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import balanced_accuracy_score
from imblearn.metrics import classification_report_imbalanced


## Logistic Regression

In [117]:
classifier = LogisticRegression(solver='lbfgs',
                                max_iter=200,
                                random_state=42)

result=classifier.fit(X_train_scaled, y_train)

In [119]:
ml_df=pd.DataFrame(columns=X.columns.tolist())

log_coefs=list(zip(X.columns.tolist(),
                   result.coef_.tolist()[0]))
log_coefs.sort(key = lambda x:abs(x[1]),reverse=True)
print(log_coefs)

[('b_points', -0.1824615414535354), ('a_points', 0.15511926606915202), ('b_penalty', -0.06564982149646481), ('a_penalty', -0.02749780431576332), ('b_incomplete_pass', 0.023906028869673343), ('surface', -0.023085189859557657), ('a_incomplete_pass', 0.017281618405634802), ('a_rush', -0.016698698877442287), ('b_penalty_yards', 0.015049118391694592), ('b_pass', -0.014822322245921), ('a_offense_rush_yards', 0.004773773927400418), ('b_rush', 0.003949837695049864), ('a_penalty_yards', 0.003607549106838031), ('a_offense_pass_yards', 0.003408438698589183), ('b_offense_rush_yards', 0.0003674339227463227), ('b_offense_pass_yards', -0.0003440425764299908), ('a_pass', -5.7104848581939025e-05)]


In [120]:
y_pred = classifier.predict(X_test)
results = pd.DataFrame({"Prediction": y_pred, "Actual": y_test}).reset_index(drop=True)
print(classifier.get_params())
results.head(10)

{'C': 1.0, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 200, 'multi_class': 'auto', 'n_jobs': None, 'penalty': 'l2', 'random_state': 42, 'solver': 'lbfgs', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}


,Prediction,Actual
0,0,0
1,1,1
2,0,0
3,1,1
4,1,1
5,0,0
6,1,1
7,1,0
8,1,1
9,1,1


In [122]:
logResAcc=balanced_accuracy_score(y_test, y_pred)
print('balanced accuracy score = ',accuracy_score(y_test, y_pred))

# Display the confusion matrix
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, y_pred)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual B", "Actual A"], columns=["Predicted B", "Predicted A"])

print("")
print(cm_df)

# Print the imbalanced classification report
print("")
print(classification_report_imbalanced(y_test, y_pred))

balanced accuracy score =  0.7260950030845157

          Predicted B  Predicted A
Actual B          521          246
Actual A          198          656

                   pre       rec       spe        f1       geo       iba       sup

          0       0.72      0.68      0.77      0.70      0.72      0.52       767
          1       0.73      0.77      0.68      0.75      0.72      0.53       854

avg / total       0.73      0.73      0.72      0.73      0.72      0.52      1621



# Balanced Random Forest Classifiers

In [123]:
# Resample the training data with the BalancedRandomForestClassifier
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=50, random_state=42) 

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Making predictions using the testing data.
y_pred = rf_model.predict(X_test_scaled)

In [124]:
brfcAcc=balanced_accuracy_score(y_test, y_pred)
print('balanced accuracy score = ',accuracy_score(y_test, y_pred))

# Display the confusion matrix
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, y_pred)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual B", "Actual A"], columns=["Predicted B", "Predicted A"])

print("")
print(cm_df)

# Print the imbalanced classification report
print("")
print(classification_report_imbalanced(y_test, y_pred))

balanced accuracy score =  0.7075879086983343

          Predicted B  Predicted A
Actual B          541          226
Actual A          248          606

                   pre       rec       spe        f1       geo       iba       sup

          0       0.69      0.71      0.71      0.70      0.71      0.50       767
          1       0.73      0.71      0.71      0.72      0.71      0.50       854

avg / total       0.71      0.71      0.71      0.71      0.71      0.50      1621

